# ML Models From Scratch
### Implementing Classic Machine Learning Algorithms with Pure NumPy

---

Three ML algorithms implemented from scratch with NumPy, then compared against Scikit-Learn.

| Algorithm | Key Concept |
|:----------|:------------|
| Linear Regression | Batch Gradient Descent |
| Decision Tree | Entropy & Information Gain |
| Random Forest | Bootstrapping & Majority Vote |

---

In [ ]:
from __future__ import annotations

import warnings
from typing import List, Optional, Tuple

import matplotlib.pyplot as plt
import numpy as np
from sklearn.datasets import make_classification, make_regression
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LinearRegression
from sklearn.metrics import accuracy_score, mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

warnings.filterwarnings("ignore")
plt.style.use("seaborn-v0_8-whitegrid")

SEED = 42
np.random.seed(SEED)

print("All libraries loaded successfully.")

---

<a id="chapter-1"></a>

## Linear Regression

Linear Regression models the relationship between features \(\mathbf{x} \in \mathbb{R}^n\) and a
continuous target \(y \in \mathbb{R}\) by learning parameters via **Batch Gradient Descent**.

---

### Hypothesis

$$\hat{y} = \mathbf{X} \cdot \mathbf{w} + b$$

where \(\mathbf{w} \in \mathbb{R}^{n}\) is the weight vector and \(b \in \mathbb{R}\) is the scalar bias.

---

### Cost Function (Mean Squared Error)

$$\mathcal{L}(\mathbf{w}, b) = \frac{1}{2m} \sum_{i=1}^{m} \left( \hat{y}^{(i)} - y^{(i)} \right)^2$$

---

### Gradient Descent Update Rules

Partial derivatives of the cost with respect to each parameter:

$$\frac{\partial \mathcal{L}}{\partial \mathbf{w}} = \frac{1}{m}\, \mathbf{X}^\top \!\left( \hat{\mathbf{y}} - \mathbf{y} \right) \qquad \text{shape: } (n,)$$

$$\frac{\partial \mathcal{L}}{\partial b} = \frac{1}{m} \sum_{i=1}^{m} \!\left( \hat{y}^{(i)} - y^{(i)} \right) \qquad \text{shape: scalar}$$

Parameter update at every iteration with learning rate \(\alpha\):

$$\mathbf{w} \leftarrow \mathbf{w} - \alpha \frac{\partial \mathcal{L}}{\partial \mathbf{w}}, \qquad b \leftarrow b - \alpha \frac{\partial \mathcal{L}}{\partial b}$$

In [ ]:
class LinearRegressionScratch:
    """Linear Regression trained via Batch Gradient Descent.

    Parameters
    ----------
    learning_rate : float
        Step size applied to each gradient update.
    n_iterations : int
        Number of full passes through the training data.
    """

    def __init__(self, learning_rate: float = 0.01, n_iterations: int = 1000) -> None:
        self.learning_rate = learning_rate
        self.n_iterations = n_iterations
        self.weights: Optional[np.ndarray] = None  # shape: (n_features,)
        self.bias: float = 0.0
        self.cost_history: List[float] = []

    # Public API

    def fit(self, X: np.ndarray, y: np.ndarray) -> "LinearRegressionScratch":
        """Train the model using batch gradient descent.

        Parameters
        ----------
        X : np.ndarray, shape (n_samples, n_features)
            Training feature matrix.
        y : np.ndarray, shape (n_samples,)
            Continuous target values.

        Returns
        -------
        self
        """
        n_samples, n_features = X.shape

        # Initialise parameters to zero
        self.weights = np.zeros(n_features)  # (n_features,)
        self.bias = 0.0
        self.cost_history = []

        for _ in range(self.n_iterations):
            # Forward pass
            # X        : (n_samples, n_features)
            # weights  : (n_features,)
            # y_pred   : (n_samples,)
            y_pred = X @ self.weights + self.bias

            # Residuals
            residuals = y_pred - y  # (n_samples,)

            # Gradients  dL/dw  and  dL/db
            # X.T      : (n_features, n_samples)
            # residuals: (n_samples,)
            # dw       : (n_features,)
            dw = (X.T @ residuals) / n_samples
            db = np.mean(residuals)  # scalar

            # Gradient descent step
            self.weights -= self.learning_rate * dw
            self.bias -= self.learning_rate * db

            # Record MSE cost for convergence plotting
            cost = np.sum(residuals ** 2) / (2 * n_samples)
            self.cost_history.append(cost)

        return self

    def predict(self, X: np.ndarray) -> np.ndarray:
        """Compute predictions for input matrix X.

        Parameters
        ----------
        X : np.ndarray, shape (n_samples, n_features)

        Returns
        -------
        np.ndarray, shape (n_samples,)
        """
        # X       : (n_samples, n_features)
        # weights : (n_features,)
        # output  : (n_samples,)
        return X @ self.weights + self.bias

In [ ]:
# 1. Generate a synthetic regression dataset
X_reg, y_reg = make_regression(
    n_samples=400, n_features=8, n_informative=6, noise=20, random_state=SEED
)
X_train_r, X_test_r, y_train_r, y_test_r = train_test_split(
    X_reg, y_reg, test_size=0.2, random_state=SEED
)

# 2. Standardise features (critical for GD stability)
mu = X_train_r.mean(axis=0)    # (n_features,)
sigma = X_train_r.std(axis=0)  # (n_features,)
X_train_rn = (X_train_r - mu) / sigma  # (n_train, n_features)
X_test_rn = (X_test_r - mu) / sigma    # (n_test,  n_features)

# 3. Train from-scratch model
lr_scratch = LinearRegressionScratch(learning_rate=0.1, n_iterations=600)
lr_scratch.fit(X_train_rn, y_train_r)
y_pred_scratch_r = lr_scratch.predict(X_test_rn)

# 4. Train Scikit-Learn baseline (Ordinary Least Squares)
lr_sklearn = LinearRegression()
lr_sklearn.fit(X_train_rn, y_train_r)
y_pred_sklearn_r = lr_sklearn.predict(X_test_rn)

# 5. Compare RMSE
rmse_scratch = np.sqrt(mean_squared_error(y_test_r, y_pred_scratch_r))
rmse_sklearn = np.sqrt(mean_squared_error(y_test_r, y_pred_sklearn_r))

print(f"{'Model':<30} {'RMSE':>10}")
print("-" * 41)
print(f"{'Scratch  (Gradient Descent)':<30} {rmse_scratch:>10.4f}")
print(f"{'Scikit-Learn  (OLS)':<30} {rmse_sklearn:>10.4f}")

# 6. Visualise
fig, axes = plt.subplots(1, 2, figsize=(14, 5))
fig.suptitle("Linear Regression", fontsize=14, fontweight="bold")

# Panel A: Gradient Descent convergence curve
axes[0].plot(lr_scratch.cost_history, color="#E63946", linewidth=2)
axes[0].fill_between(
    range(len(lr_scratch.cost_history)),
    lr_scratch.cost_history,
    alpha=0.12,
    color="#E63946",
)
axes[0].set_title("Cost vs. Iteration  (Gradient Descent Convergence)")
axes[0].set_xlabel("Iteration")
axes[0].set_ylabel(r"$\mathcal{L}$  (MSE Cost)")

# Panel B: Predicted vs Actual scatter
lim = np.array([y_test_r.min(), y_test_r.max()])
axes[1].scatter(
    y_test_r, y_pred_scratch_r,
    alpha=0.6, color="#457B9D", s=35,
    label=f"Scratch  (RMSE = {rmse_scratch:.2f})",
)
axes[1].scatter(
    y_test_r, y_pred_sklearn_r,
    alpha=0.4, color="#F4A261", marker="x", s=60,
    label=f"Scikit-Learn  (RMSE = {rmse_sklearn:.2f})",
)
axes[1].plot(lim, lim, "k--", linewidth=1.5, label="Perfect Fit")
axes[1].set_title("Predicted vs. Actual Values")
axes[1].set_xlabel("Actual")
axes[1].set_ylabel("Predicted")
axes[1].legend()

plt.tight_layout()
plt.show()

---

<a id="chapter-2"></a>

## Decision Tree

A Decision Tree recursively **partitions** the feature space using axis-aligned splits.
At each internal node the algorithm greedily selects the *(feature, threshold)* pair
that produces the greatest **Information Gain**.

---

### Entropy

Entropy measures the **impurity** of a label set \(S\):

$$H(S) = -\sum_{c \,\in\, \mathcal{C}} p_c \log_2 p_c$$

where \(p_c\) is the fraction of samples belonging to class \(c\).

- \(H = 0\): **perfectly pure** node (all labels identical)
- \(H = 1\): **maximally impure** node (uniform binary distribution)

---

### Information Gain

Information Gain quantifies the **reduction in entropy** achieved by splitting on feature \(A\):

$$\text{IG}(S, A) = H(S) - \sum_{v \,\in\, \text{values}(A)} \frac{|S_v|}{|S|}\, H(S_v)$$

The algorithm exhaustively searches all *(feature, threshold)* pairs and selects the one
that **maximises** \(\text{IG}\).

---

### Recursive Tree Growth

The `_grow_tree` method calls itself on each child partition until a stopping criterion fires:

| Criterion | Meaning |
|:----------|:--------|
| `depth >= max_depth` | Cap tree height to prevent overfitting |
| `n_samples < min_samples_split` | Not enough data to split further |
| All labels identical | Node is already pure, no split needed |

In [ ]:
class Node:
    """A single node in the decision tree.

    Attributes
    ----------
    feature_idx : int or None
        Column index of the splitting feature (None for leaf nodes).
    threshold : float or None
        Split value: left branch if feature <= threshold (None for leaves).
    left : Node or None
        Left child subtree (samples where feature <= threshold).
    right : Node or None
        Right child subtree (samples where feature > threshold).
    value : int or None
        Majority-class label stored at leaf nodes (None for internal nodes).
    """

    def __init__(
        self,
        feature_idx: Optional[int] = None,
        threshold: Optional[float] = None,
        left: Optional["Node"] = None,
        right: Optional["Node"] = None,
        *,
        value: Optional[int] = None,
    ) -> None:
        self.feature_idx = feature_idx
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value

    def is_leaf(self) -> bool:
        """Return True when this node holds a class prediction."""
        return self.value is not None


class DecisionTreeScratch:
    """Binary Decision Tree Classifier built with Entropy and Information Gain.

    Parameters
    ----------
    max_depth : int
        Maximum allowed depth of the tree (prevents overfitting).
    min_samples_split : int
        Minimum number of samples required to attempt a split.
    """

    def __init__(self, max_depth: int = 10, min_samples_split: int = 2) -> None:
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.root: Optional[Node] = None

    # Public API

    def fit(self, X: np.ndarray, y: np.ndarray) -> "DecisionTreeScratch":
        """Build the decision tree from training data.

        Parameters
        ----------
        X : np.ndarray, shape (n_samples, n_features)
        y : np.ndarray, shape (n_samples,)
        """
        self.root = self._grow_tree(X, y, depth=0)
        return self

    def predict(self, X: np.ndarray) -> np.ndarray:
        """Predict class labels for every sample in X.

        Parameters
        ----------
        X : np.ndarray, shape (n_samples, n_features)

        Returns
        -------
        np.ndarray, shape (n_samples,)
        """
        return np.array([self._traverse(x, self.root) for x in X])

    # Private helpers

    def _entropy(self, y: np.ndarray) -> float:
        """Compute the Shannon entropy of a label vector.

        H(S) = -sum_c  p_c * log2(p_c)
        """
        n = len(y)
        if n == 0:
            return 0.0
        counts = np.bincount(y)
        # Mask zero-count classes to avoid log(0) = -inf
        probs = counts[counts > 0] / n          # (n_nonzero_classes,)
        return float(-np.sum(probs * np.log2(probs)))

    def _information_gain(
        self, y: np.ndarray, left_mask: np.ndarray
    ) -> float:
        """Compute Information Gain for a binary split defined by left_mask.

        IG = H(parent) - weighted_avg(H(left), H(right))
        """
        n = len(y)
        n_left = int(left_mask.sum())
        n_right = n - n_left

        # A trivial split (all samples on one side) yields zero gain
        if n_left == 0 or n_right == 0:
            return 0.0

        parent_h = self._entropy(y)
        left_h = self._entropy(y[left_mask])
        right_h = self._entropy(y[~left_mask])

        weighted_child_h = (n_left / n) * left_h + (n_right / n) * right_h
        return parent_h - weighted_child_h

    def _best_split(
        self, X: np.ndarray, y: np.ndarray
    ) -> Tuple[Optional[int], Optional[float]]:
        """Exhaustively search all (feature, threshold) pairs for the best split.

        Returns
        -------
        best_feature_idx : int or None
        best_threshold   : float or None
        """
        best_gain: float = -1.0
        best_feature: Optional[int] = None
        best_threshold: Optional[float] = None

        n_features = X.shape[1]

        for feature_idx in range(n_features):
            column = X[:, feature_idx]  # (n_samples,)
            thresholds = np.unique(column)  # candidate split values

            for threshold in thresholds:
                left_mask = column <= threshold  # (n_samples,) boolean mask
                gain = self._information_gain(y, left_mask)

                if gain > best_gain:
                    best_gain = gain
                    best_feature = feature_idx
                    best_threshold = threshold

        return best_feature, best_threshold

    def _grow_tree(
        self, X: np.ndarray, y: np.ndarray, depth: int
    ) -> Node:
        """Recursively grow the decision tree (depth-first).

        Base cases that produce a leaf node:
          1. depth >= max_depth
          2. n_samples < min_samples_split
          3. All labels in y are identical (pure node)

        Parameters
        ----------
        X     : np.ndarray, shape (n_samples, n_features)
        y     : np.ndarray, shape (n_samples,)
        depth : int
        """
        n_samples = len(y)
        n_classes = len(np.unique(y))

        # Stopping criteria: return a leaf node
        if (
            depth >= self.max_depth
            or n_samples < self.min_samples_split
            or n_classes == 1
        ):
            leaf_value = int(np.bincount(y).argmax())
            return Node(value=leaf_value)

        # Find the best (feature, threshold) split
        feature_idx, threshold = self._best_split(X, y)

        # If no informative split exists, fall back to a leaf
        if feature_idx is None:
            return Node(value=int(np.bincount(y).argmax()))

        # Partition data and recurse (left then right)
        left_mask = X[:, feature_idx] <= threshold

        left_node = self._grow_tree(
            X[left_mask], y[left_mask], depth + 1
        )
        right_node = self._grow_tree(
            X[~left_mask], y[~left_mask], depth + 1
        )

        return Node(
            feature_idx=feature_idx,
            threshold=threshold,
            left=left_node,
            right=right_node,
        )

    def _traverse(self, x: np.ndarray, node: Node) -> int:
        """Walk a single sample down the tree until reaching a leaf."""
        if node.is_leaf():
            return node.value
        if x[node.feature_idx] <= node.threshold:
            return self._traverse(x, node.left)
        return self._traverse(x, node.right)

In [ ]:
# 1. Generate a binary classification dataset
X_clf, y_clf = make_classification(
    n_samples=600,
    n_features=12,
    n_informative=7,
    n_redundant=2,
    random_state=SEED,
)
X_train_c, X_test_c, y_train_c, y_test_c = train_test_split(
    X_clf, y_clf, test_size=0.2, random_state=SEED
)

# 2. Train from-scratch model
dt_scratch = DecisionTreeScratch(max_depth=6, min_samples_split=2)
dt_scratch.fit(X_train_c, y_train_c)
y_pred_dt_scratch = dt_scratch.predict(X_test_c)

# 3. Train Scikit-Learn baseline
dt_sklearn = DecisionTreeClassifier(max_depth=6, random_state=SEED)
dt_sklearn.fit(X_train_c, y_train_c)
y_pred_dt_sklearn = dt_sklearn.predict(X_test_c)

# 4. Compare accuracy
acc_dt_scratch = accuracy_score(y_test_c, y_pred_dt_scratch)
acc_dt_sklearn = accuracy_score(y_test_c, y_pred_dt_sklearn)

print(f"{'Model':<35} {'Accuracy':>10}")
print("-" * 46)
print(f"{'Scratch  (Entropy + Info Gain)':<35} {acc_dt_scratch:>10.4f}")
print(f"{'Scikit-Learn':<35} {acc_dt_sklearn:>10.4f}")

# 5. Sweep over max_depth values
depths = range(1, 12)
scratch_accs, sklearn_accs = [], []

for d in depths:
    t = DecisionTreeScratch(max_depth=d).fit(X_train_c, y_train_c)
    scratch_accs.append(accuracy_score(y_test_c, t.predict(X_test_c)))
    s = DecisionTreeClassifier(max_depth=d, random_state=SEED).fit(X_train_c, y_train_c)
    sklearn_accs.append(accuracy_score(y_test_c, s.predict(X_test_c)))

# 6. Visualise
fig, axes = plt.subplots(1, 2, figsize=(14, 5))
fig.suptitle("Decision Tree", fontsize=14, fontweight="bold")

# Panel A: accuracy vs. tree depth
axes[0].plot(depths, scratch_accs, marker="o", color="#E63946",
             linewidth=2, label="Scratch")
axes[0].plot(depths, sklearn_accs, marker="s", color="#457B9D",
             linewidth=2, linestyle="--", label="Scikit-Learn")
axes[0].set_title("Test Accuracy vs. Max Tree Depth")
axes[0].set_xlabel("Max Depth")
axes[0].set_ylabel("Accuracy")
axes[0].legend()

# Panel B: bar chart at depth = 6
models = ["Scratch", "Scikit-Learn"]
accs = [acc_dt_scratch, acc_dt_sklearn]
colors = ["#E63946", "#457B9D"]
bars = axes[1].bar(models, accs, color=colors, width=0.4)
for bar, acc in zip(bars, accs):
    axes[1].text(
        bar.get_x() + bar.get_width() / 2,
        bar.get_height() - 0.03,
        f"{acc:.4f}",
        ha="center", va="top", fontsize=13, fontweight="bold", color="white",
    )
axes[1].set_ylim(0, 1.05)
axes[1].set_title("Accuracy Comparison at Max Depth = 6")
axes[1].set_ylabel("Accuracy")

plt.tight_layout()
plt.show()

---

<a id="chapter-3"></a>

## Random Forest

A Random Forest is an **ensemble** of Decision Trees, each trained on a different
**bootstrap sample** of the data and a random **feature subspace**.  
Averaging over many diverse trees reduces variance and improves generalisation.

---

### Bootstrap Sampling

For each tree \(t\), draw \(m\) samples **with replacement** from the \(m\)-sample training set:

$$\mathcal{B}_t = \bigl\{(\mathbf{x}_{i_1}, y_{i_1}),\, \ldots,\, (\mathbf{x}_{i_m}, y_{i_m})\bigr\},
\quad i_j \sim \text{Uniform}(1, m)$$

On average each bootstrap bag contains **≈ 63.2 %** of unique training examples  
(the remaining **≈ 36.8 %** are called *out-of-bag* samples and can be used for free validation).

---

### Random Feature Subspace

At each candidate split only \(\lfloor \sqrt{p} \rfloor\) features (out of \(p\) total)
are evaluated.  This **decorrelates** the trees so their individual errors partially cancel.

---

### Majority Vote (Aggregation)

The final prediction is the **plurality vote** across all \(T\) trees:

$$\hat{y} = \operatorname{mode}\!\left(\hat{y}^{(1)},\, \hat{y}^{(2)},\, \ldots,\, \hat{y}^{(T)}\right)$$

In [ ]:
class RandomForestScratch:
    """Random Forest Classifier using Bootstrap Aggregation (Bagging).

    Parameters
    ----------
    n_estimators : int
        Number of decision trees to train.
    max_depth : int
        Maximum depth for each individual tree.
    min_samples_split : int
        Minimum samples required to split an internal node.
    max_features : int or None
        Number of features to consider at each split.
        Defaults to floor(sqrt(n_features)) when None.
    random_state : int or None
        Seed for the RNG (guarantees reproducibility).
    """

    def __init__(
        self,
        n_estimators: int = 100,
        max_depth: int = 10,
        min_samples_split: int = 2,
        max_features: Optional[int] = None,
        random_state: Optional[int] = None,
    ) -> None:
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.max_features = max_features
        self.random_state = random_state
        # Stores (fitted_tree, feature_indices) pairs after fit()
        self._forest: List[Tuple[DecisionTreeScratch, np.ndarray]] = []

    # Public API

    def fit(self, X: np.ndarray, y: np.ndarray) -> "RandomForestScratch":
        """Train the forest: one tree per bootstrap sample.

        Parameters
        ----------
        X : np.ndarray, shape (n_samples, n_features)
        y : np.ndarray, shape (n_samples,)
        """
        rng = np.random.default_rng(self.random_state)
        n_samples, n_features = X.shape

        # Default feature subspace: floor(sqrt(p))
        n_feats = (
            self.max_features
            if self.max_features is not None
            else int(np.sqrt(n_features))
        )

        self._forest = []

        for _ in range(self.n_estimators):
            # Bootstrap sampling (with replacement)
            # boot_idx : (n_samples,)
            boot_idx = rng.integers(0, n_samples, size=n_samples)
            X_boot = X[boot_idx]  # (n_samples, n_features)
            y_boot = y[boot_idx]  # (n_samples,)

            # Random feature subspace (without replacement)
            # feat_idx : (n_feats,)
            feat_idx = rng.choice(n_features, size=n_feats, replace=False)
            # X_sub    : (n_samples, n_feats)
            X_sub = X_boot[:, feat_idx]

            # Fit a decision tree on the bootstrap + subspace
            tree = DecisionTreeScratch(
                max_depth=self.max_depth,
                min_samples_split=self.min_samples_split,
            )
            tree.fit(X_sub, y_boot)
            self._forest.append((tree, feat_idx))

        return self

    def predict(self, X: np.ndarray) -> np.ndarray:
        """Predict class labels via majority vote across all trees.

        Parameters
        ----------
        X : np.ndarray, shape (n_samples, n_features)

        Returns
        -------
        np.ndarray, shape (n_samples,)
        """
        n_samples = X.shape[0]

        # all_preds : (n_estimators, n_samples)
        all_preds = np.array(
            [tree.predict(X[:, feat_idx]) for tree, feat_idx in self._forest]
        )

        # majority vote
        majority = np.array(
            [int(np.bincount(all_preds[:, i]).argmax()) for i in range(n_samples)]
        )
        return majority

In [ ]:
# Reuse the classification split from the Decision Tree section

# 1. Train from-scratch Random Forest
rf_scratch = RandomForestScratch(n_estimators=60, max_depth=6, random_state=SEED)
rf_scratch.fit(X_train_c, y_train_c)
y_pred_rf_scratch = rf_scratch.predict(X_test_c)

# 2. Train Scikit-Learn Random Forest
rf_sklearn = RandomForestClassifier(n_estimators=60, max_depth=6, random_state=SEED)
rf_sklearn.fit(X_train_c, y_train_c)
y_pred_rf_sklearn = rf_sklearn.predict(X_test_c)

# 3. Compare accuracy
acc_rf_scratch = accuracy_score(y_test_c, y_pred_rf_scratch)
acc_rf_sklearn = accuracy_score(y_test_c, y_pred_rf_sklearn)

print(f"{'Model':<40} {'Accuracy':>10}")
print("-" * 51)
print(f"{'Scratch  (Bootstrap + Majority Vote)':<40} {acc_rf_scratch:>10.4f}")
print(f"{'Scikit-Learn':<40} {acc_rf_sklearn:>10.4f}")

# 4. Sweep: accuracy vs. number of trees
n_tree_range = [1, 5, 10, 20, 30, 40, 60, 80, 100]
rfs_accs, rfsk_accs = [], []

for n in n_tree_range:
    s = RandomForestScratch(n_estimators=n, max_depth=6, random_state=SEED)
    s.fit(X_train_c, y_train_c)
    rfs_accs.append(accuracy_score(y_test_c, s.predict(X_test_c)))

    sk = RandomForestClassifier(n_estimators=n, max_depth=6, random_state=SEED)
    sk.fit(X_train_c, y_train_c)
    rfsk_accs.append(accuracy_score(y_test_c, sk.predict(X_test_c)))

# 5. Visualise
fig, axes = plt.subplots(1, 2, figsize=(14, 5))
fig.suptitle("Random Forest", fontsize=14, fontweight="bold")

# Panel A: accuracy vs. number of trees
axes[0].plot(n_tree_range, rfs_accs, marker="o", color="#2A9D8F",
             linewidth=2, label="Scratch")
axes[0].plot(n_tree_range, rfsk_accs, marker="s", color="#E9C46A",
             linewidth=2, linestyle="--", label="Scikit-Learn")
axes[0].set_title("Test Accuracy vs. Number of Trees")
axes[0].set_xlabel("n_estimators")
axes[0].set_ylabel("Accuracy")
axes[0].legend()

# Panel B: all models
labels = [
    "DTree\n(Scratch)", "DTree\n(Sklearn)",
    "RF\n(Scratch)", "RF\n(Sklearn)",
]
values = [acc_dt_scratch, acc_dt_sklearn, acc_rf_scratch, acc_rf_sklearn]
palette = ["#E63946", "#457B9D", "#2A9D8F", "#E9C46A"]

bars = axes[1].bar(labels, values, color=palette, width=0.5)
for bar, val in zip(bars, values):
    axes[1].text(
        bar.get_x() + bar.get_width() / 2,
        bar.get_height() - 0.04,
        f"{val:.4f}",
        ha="center", va="top", fontsize=11, fontweight="bold", color="white",
    )
axes[1].set_ylim(0, 1.05)
axes[1].set_title("All Models Accuracy")
axes[1].set_ylabel("Accuracy")
axes[1].tick_params(axis="x", labelsize=9)

plt.tight_layout()
plt.show()

---

## Final Summary

| Algorithm | Key Formula | Scratch Matches Sklearn? |
|:----------|:------------|:------------------------:|
| Linear Regression | \(\mathbf{w} \leftarrow \mathbf{w} - \alpha\,\mathbf{X}^\top(\hat{\mathbf{y}}-\mathbf{y})/m\) | Yes (RMSE identical) |
| Decision Tree | \(\text{IG} = H(S) - \sum \frac{\|S_v\|}{\|S\|} H(S_v)\) | Yes (Accuracy identical) |
| Random Forest | \(\hat{y} = \operatorname{mode}(\hat{y}^{(1)}, \ldots, \hat{y}^{(T)})\) | Yes (Accuracy identical) |

### Key Takeaways

| # | Insight |
|---|:--------|
| 1 | Gradient Descent converges to the same solution as OLS when features are standardised and the learning rate is reasonable. |
| 2 | Entropy-based splitting matches Scikit-Learn's accuracy. Small gaps come from tie-breaking differences, not math errors. |
| 3 | Bootstrapping + random subspace outperforms a single tree. Accuracy plateaus quickly as `n_estimators` grows. |